In [1]:
# -*- coding: utf-8 -*-
"""Longitudinal Anxiety Intervention Analysis with Time Series Techniques

This notebook adapts the MoE framework to incorporate time series analysis techniques.
It examines the longitudinal effects of the intervention on anxiety levels, tracking
changes over time and identifying potential delayed or long-term impacts.

Workflow:
1. Data Loading and Validation: Load synthetic anxiety intervention time series data, validate its structure, content, and data types. Handle potential errors gracefully.
2. Time Series Analysis (Placeholder): Implement a placeholder for time series analysis, with clear steps for future expansion, including specific algorithms and validation methods.
3. Data Visualization: Generate Time Series Line Plots, Parallel Coordinates, and Hypergraph plots, with detailed explanations and error handling for visualization issues.
4. Statistical Summary: Perform bootstrap analysis and generate summary statistics, including validation of results and handling of potential statistical errors.
5. LLM Insights Report: Synthesize findings using Grok, Claude, and Grok-Enhanced, emphasizing time series insights, validating LLM outputs, and handling potential LLM API errors.

Keywords: Time Series Analysis, Longitudinal Data, Anxiety Intervention, Temporal Effects, LLMs, Data Visualization
"""

# Suppress warnings (with caution - better to handle specific warnings)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="plotly")

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import shap
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from io import StringIO
import plotly.express as px
from scipy.stats import bootstrap, shapiro  # Import shapiro for normality test
from matplotlib.colors import LinearSegmentedColormap

# Google Colab environment check
try:
    from google.colab import drive
    drive.mount("/content/drive")
    COLAB_ENV = True
except Exception as e:  # Catch any exception during drive mount
    COLAB_ENV = False
    print(f"Not running in Google Colab environment or Drive mounting failed: {e}")

# Constants
OUTPUT_PATH = "./output_anxiety_time_series/" if not COLAB_ENV else "/content/drive/MyDrive/output_anxiety_time_series/"
PARTICIPANT_ID_COLUMN = "participant_id"
GROUP_COLUMN = "group"
ANXIETY_PRE_COLUMN = "anxiety_pre"
ANXIETY_POST_COLUMN = "anxiety_post"
ANXIETY_WEEK1_COLUMN = "anxiety_week1"
ANXIETY_WEEK2_COLUMN = "anxiety_week2"
ANXIETY_WEEK3_COLUMN = "anxiety_week3"
TIME_SERIES_COLUMNS = [ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, ANXIETY_WEEK1_COLUMN, ANXIETY_WEEK2_COLUMN, ANXIETY_WEEK3_COLUMN]
MODEL_GROK_NAME = "grok-base"
MODEL_CLAUDE_NAME = "claude-3.7-sonnet"
MODEL_GROK_ENHANCED_NAME = "grok-enhanced"
LINE_WIDTH = 2.5
BOOTSTRAP_RESAMPLES = 500  # Define number of bootstrap resamples
NEON_COLORS = ["#FF00FF", "#00FFFF", "#FFFF00", "#00FF00"] # Visualization colors

# Placeholder API Keys (Security Warning)
GROK_API_KEY = "YOUR_GROK_API_KEY"  # Placeholder
CLAUDE_API_KEY = "YOUR_CLAUDE_API_KEY" # Placeholder

# --- DDQN Agent Class ---
class DDQNAgent:
    """
    A simplified DDQN agent for demonstration purposes.  This is a *placeholder*
    and would need significant adaptation for a real-world application.
    """
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        # Initialize Q-network and target network with random values (for demonstration)
        self.q_network = np.random.rand(state_dim, action_dim)
        self.target_network = np.copy(self.q_network)

    def act(self, state, epsilon=0.01):
        """Epsilon-greedy action selection."""
        if np.random.rand() < epsilon:
            return np.random.choice(self.action_dim)  # Explore
        else:
            return np.argmax(self.q_network[state])  # Exploit

    def learn(self, batch, gamma=0.99, learning_rate=0.1):
        """Placeholder learning function.  A real implementation would update the Q-network."""
        for state, action, reward, next_state in batch:
            # Simplified DDQN update (replace with actual update rule)
            q_target = reward + gamma * np.max(self.target_network[next_state])
            q_predict = self.q_network[state, action]
            self.q_network[state, action] += learning_rate * (q_target - q_predict)

    def update_target_network(self):
        """Placeholder target network update."""
        self.target_network = np.copy(self.q_network)


# --- Functions ---

def create_output_directory(path):
    """Creates the output directory if it doesn't exist, handling potential errors."""
    try:
        os.makedirs(path, exist_ok=True)
    except OSError as e:
        print(f"Error creating output directory: {e}")
        return False  # Indicate failure
    return True  # Indicate success

def load_data_from_synthetic_string(csv_string):
    """Loads data from a CSV string, handling potential read errors."""
    try:
        csv_file = StringIO(csv_string)
        return pd.read_csv(csv_file)
    except pd.errors.ParserError as e:
        print(f"Error parsing CSV data: {e}")
        return None  # Return None to indicate failure
    except Exception as e:
        print(f"An unexpected error occurred during data loading: {e}")
        return None

def validate_dataframe(df, required_columns):
    """Validates the DataFrame: checks for missing columns, non-numeric data,
    duplicate participant IDs, valid group labels, and plausible anxiety ranges.
    Returns a tuple: (True/False for validity, valid_groups or None).
    """
    if df is None:  # Check if DataFrame is valid
        print("Error: DataFrame is None. Cannot validate.")
        return False, None

    # 1. Check for Missing Columns
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"Error: Missing columns: {missing_columns}")
        return False, None

    # 2. Check for Non-Numeric Values
    for col in required_columns:
        if col != PARTICIPANT_ID_COLUMN and col != GROUP_COLUMN:
            if not pd.api.types.is_numeric_dtype(df[col]):
                print(f"Error: Non-numeric values found in column: {col}")
                return False, None

    # 3. Check for Duplicate Participant IDs
    if df[PARTICIPANT_ID_COLUMN].duplicated().any():
        print("Error: Duplicate participant IDs found.")
        return False, None

    # 4. Check Group Labels
    valid_groups = ["Group A", "Group B", "Control"]  # Define valid group names
    invalid_groups = df[~df[GROUP_COLUMN].isin(valid_groups)][GROUP_COLUMN].unique()
    if invalid_groups.size > 0:
        print(f"Error: Invalid group labels found: {invalid_groups}")
        return False, None

    # 5. Range Checks for Anxiety Scores (assuming a scale of 0-10)
    for col in TIME_SERIES_COLUMNS:
        if df[col].min() < 0 or df[col].max() > 10:
            print(f"Error: Anxiety scores in column '{col}' are out of range (0-10).")
            return False, None

    return True, valid_groups

def analyze_text_with_llm(text, model_name):
    """Placeholder for LLM analysis."""
    text_lower = text.lower()
    if model_name == MODEL_GROK_NAME:
        if "time series analysis" in text_lower: return "Grok-base: Time series analysis shows trends over time, with Group A showing a sustained reduction and Group B showing an initial reduction followed by a slight increase."
        elif "line plot" in text_lower: return "Grok-base: Line plot visualizes anxiety trends over time, clearly showing the different trajectories for each group."
        else: return f"Grok-base: General analysis on '{text}'."
    elif model_name == MODEL_CLAUDE_NAME:
        if "time series analysis" in text_lower: return "Claude 3.7: Time series analysis reveals longitudinal effects, highlighting the sustained reduction in anxiety for Group A and the more variable response in Group B."
        elif "parallel coordinates" in text_lower: return "Claude 3.7: Parallel coordinates show temporal trajectories, allowing for easy comparison of individual participant responses across the different time points."
        else: return f"Claude 3.7: Enhanced time series analysis on '{text}'."
    elif model_name == MODEL_GROK_ENHANCED_NAME:
        if "time series analysis" in text_lower: return "Grok-Enhanced: Time series analysis provides nuanced insights into long-term intervention impacts, revealing distinct patterns of sustained reduction, initial improvement followed by relapse, and stability across the groups."
        elif "hypergraph" in text_lower: return "Grok-Enhanced: Hypergraph highlights participant clusters based on temporal anxiety patterns, identifying groups with similar response trajectories."
        else: return f"Grok-Enhanced: In-depth temporal analysis on '{text}'."
    return f"Model '{model_name}' not supported."

def scale_data(df, columns):
    """Scales specified columns using MinMaxScaler, handling potential errors."""
    try:
        scaler = MinMaxScaler()
        df[columns] = scaler.fit_transform(df[columns])
        return df
    except ValueError as e:
        print(f"Error during data scaling: {e}")
        return None  # Or raise the exception, depending on desired behavior
    except Exception as e:
        print(f"An unexpected error occurred during scaling: {e}")
        return None

def perform_time_series_analysis(df, output_path):
    """Placeholder function to simulate time series analysis."""
    time_series_desc = "Time Series Analysis Placeholder Output:\n"
    time_series_desc += "- Average anxiety reduction observed across all groups over time.\n"
    time_series_desc += "- Group A shows a sustained reduction in anxiety levels.\n"
    time_series_desc += "- Group B shows initial reduction, with slight increase in later weeks.\n"
    time_series_desc += "- Control group anxiety levels remain relatively stable.\n"

    print("Time Series Analysis Placeholder Output:\n" + time_series_desc)
    # In a real implementation, you would perform actual time series analysis here
    # (e.g., using ARIMA, Prophet, or other time series models) and generate
    # appropriate output and visualizations.
    return time_series_desc

def calculate_shap_values(df, feature_columns, target_column, output_path):
    """Calculates and visualizes SHAP values, handling potential errors."""
    try:
        model_rf = RandomForestRegressor(random_state=42).fit(df[feature_columns], df[target_column]) # Added random_state
        explainer = shap.TreeExplainer(model_rf)
        shap_values = explainer.shap_values(df[feature_columns])

        plt.figure(figsize=(10, 8))
        plt.style.use('dark_background')
        shap.summary_plot(shap_values, df[feature_columns], show=False, color_bar=True)
        plt.savefig(os.path.join(output_path, 'shap_summary.png'))
        plt.close()
        return f"SHAP summary for features {feature_columns} predicting {target_column}"
    except Exception as e:
        print(f"Error during SHAP value calculation: {e}")
        return "Error: SHAP value calculation failed."

def create_kde_plot(df, column1, column2, output_path, colors):
    """Creates a Kernel Density Estimate plot, handling potential errors."""
    try:
        plt.figure(figsize=(10, 6))
        plt.style.use('dark_background')
        sns.kdeplot(data=df[column1], color=colors[0], label=column1.capitalize(), linewidth=LINE_WIDTH)
        sns.kdeplot(data=df[column2], color=colors[1], label=column2.capitalize(), linewidth=LINE_WIDTH)
        plt.title('KDE Plot of Anxiety Levels', color='white')
        plt.legend(facecolor='black', edgecolor='white', labelcolor='white')
        plt.savefig(os.path.join(output_path, 'kde_plot.png'))
        plt.close()
        return f"KDE plot visualizing distributions of {column1} and {column2}"
    except KeyError as e:
        print(f"Error generating KDE plot: Column not found: {e}")
        return "Error: KDE plot generation failed.  Missing column."
    except RuntimeError as e:
        print(f"Error generating KDE plot: {e}")
        return "Error: KDE plot generation failed."
    except Exception as e:
        print(f"An unexpected error occurred while creating KDE plot: {e}")
        return "Error: KDE plot generation failed."

def create_violin_plot(df, group_column, y_column, output_path, colors):
    """Creates a violin plot, handling potential errors."""
    try:
        plt.figure(figsize=(10, 6))
        plt.style.use('dark_background')
        sns.violinplot(data=df, x=group_column, y=y_column, palette=colors, linewidth=LINE_WIDTH)
        plt.title('Violin Plot of Anxiety Distribution by Group', color='white')
        plt.savefig(os.path.join(output_path, 'violin_plot.png'))
        plt.close()
        return f"Violin plot showing {y_column} across {group_column}"
    except KeyError as e:
        print(f"Error generating violin plot: Column not found: {e}")
        return "Error: Violin plot generation failed. Missing column."
    except RuntimeError as e:
        print(f"Error generating violin plot: {e}")
        return "Error: Violin plot generation failed."
    except Exception as e:
        print(f"An unexpected error occurred while creating violin plot: {e}")
        return "Error: Violin plot generation failed."

def create_parallel_coordinates_plot(df, group_column, time_series_columns, output_path, colors):
    """Creates a parallel coordinates plot, handling potential errors."""
    try:
        plot_df = df[[group_column] + time_series_columns].copy()
        # One-Hot encode the group column *within* this function, and use that for coloring.
        plot_df = pd.get_dummies(plot_df, columns=[group_column], prefix=group_column)
        encoded_group_cols = [col for col in plot_df.columns if col.startswith(f"{group_column}_")]

        # Create a single color column based on the one-hot encoded columns
        def get_group_color(row):
            for i, col in enumerate(encoded_group_cols):
                if row[col] == 1:
                    return colors[i % len(colors)]  # Cycle through colors if needed
            return 'gray'  # Default color if no group is found (shouldn't happen)

        plot_df['color'] = plot_df[encoded_group_cols].apply(get_group_color, axis=1)

        fig = px.parallel_coordinates(plot_df, color='color', dimensions=time_series_columns, title="Anxiety Levels: Longitudinal Trajectories by Group", color_continuous_scale=px.colors.sequential.Viridis)
        fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white', title_font_size=16)
        fig.write_image(os.path.join(output_path, 'parallel_coordinates_plot_timeseries.png'))
        return f"Parallel coordinates plot of longitudinal anxiety trajectories by group"
    except KeyError as e:
        print(f"Error generating parallel coordinates plot: Column not found: {e}")
        return "Error: Parallel coordinates plot generation failed. Missing column."
    except Exception as e:
        print(f"Error generating parallel coordinates plot: {e}")
        return "Error: Parallel coordinates plot generation failed."

def visualize_hypergraph(df, anxiety_pre_column, anxiety_post_column, output_path, colors):
    """Creates a hypergraph, handling potential errors."""
    try:
        G = nx.Graph()
        participant_ids = df[PARTICIPANT_ID_COLUMN].tolist()
        G.add_nodes_from(participant_ids, bipartite=0)
        feature_sets = {
            "anxiety_pre": df[PARTICIPANT_ID_COLUMN][df[anxiety_pre_column] > df[anxiety_pre_column].mean()].tolist(),
            "anxiety_post": df[PARTICIPANT_ID_COLUMN][df[anxiety_post_column] > df[anxiety_post_column].mean()].tolist()
        }
        feature_nodes = list(feature_sets.keys())
        G.add_nodes_from(feature_nodes, bipartite=1)
        for feature, participants in feature_sets.items():
            for participant in participants:
                G.add_edge(participant, feature)
        pos = nx.bipartite_layout(G, participant_ids)
        color_map = [colors[0] if node in participant_ids else colors[1] for node in G]
        plt.figure(figsize=(12, 10))
        plt.style.use('dark_background')
        nx.draw(G, pos, with_labels=True, node_color=color_map, font_color="white", edge_color="gray", width=LINE_WIDTH, node_size=700, font_size=10)
        plt.title("Hypergraph Representation of Anxiety Patterns", color="white")
        plt.savefig(os.path.join(output_path, "hypergraph.png"))
        plt.close()
        return "Hypergraph visualizing participant relationships based on anxiety pre and post intervention"
    except KeyError as e:
        print(f"Error generating hypergraph: Column not found: {e}")
        return "Error: Hypergraph generation failed. Missing column."
    except Exception as e:
        print(f"Error generating hypergraph: {e}")
        return "Error: Hypergraph generation failed."

def create_time_series_line_plot(df, group_column, time_series_columns, output_path, colors):
    """Creates a time series line plot, handling potential errors."""
    try:
        plt.figure(figsize=(12, 6))
        plt.style.use('dark_background')

        # Group by the original group column and calculate the mean for each time point
        for i, group in enumerate(df[group_column].unique()):
            group_data = df[df[group_column] == group]
            time_points = range(len(time_series_columns))
            # Calculate the mean anxiety level for each time point
            mean_anxiety_levels = [group_data[col].mean() for col in time_series_columns]
            plt.plot(time_points, mean_anxiety_levels, label=group, color=colors[i % len(colors)], linewidth=LINE_WIDTH, marker='o')

        plt.xticks(time_points, [col.replace('anxiety_', '').capitalize() for col in time_series_columns], color='white')
        plt.xlabel('Time Point', fontsize=14, color='white')
        plt.ylabel('Mean Anxiety Level', fontsize=14, color='white')
        plt.title('Mean Anxiety Levels Over Time by Group', fontsize=16, color='white')
        plt.legend(title='Group', facecolor='black', edgecolor='white', labelcolor='white')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_path, 'time_series_line_plot.png'))
        plt.close()
        return "Time series line plot visualizing mean anxiety levels over time for each group"

    except KeyError as e:
        print(f"Error generating time series line plot: Column not found: {e}")
        return "Error: Time series line plot generation failed. Missing column."
    except Exception as e:
        print(f"Error generating time series line plot: {e}")
        return "Error: Time series line plot generation failed."

def perform_bootstrap(data, statistic, n_resamples=BOOTSTRAP_RESAMPLES):
    """Performs bootstrap resampling, calculates confidence intervals, and checks for normality, handling potential errors."""
    try:
        # Perform bootstrap
        bootstrap_result = bootstrap((data,), statistic, n_resamples=n_resamples, method='percentile', random_state=42) # Added random_state
        ci = bootstrap_result.confidence_interval

        # Check for normality of the bootstrap distribution (optional, but good practice)
        try:
            stat, p_value = shapiro(bootstrap_result.bootstrap_distribution)
            if p_value > 0.05:
                normality_message = "Bootstrap distribution appears normal."
            else:
                normality_message = "Bootstrap distribution may not be normal."
        except Exception as e:
            normality_message = f"Error during normality test: {e}"

        return ci, normality_message

    except Exception as e:
        print(f"Error during bootstrap analysis: {e}")
        return (None, None), "Error: Bootstrap analysis failed."

def save_summary(df, bootstrap_ci, output_path):
    """Saves descriptive statistics and bootstrap CI, handling potential errors."""
    try:
        summary_text = df.describe().to_string() + f"\nBootstrap CI for anxiety_post mean: {bootstrap_ci}"
        with open(os.path.join(output_path, 'summary.txt'), 'w') as f:
            f.write(summary_text)
        return summary_text
    except Exception as e:
        print(f"Error saving summary statistics: {e}")
        return "Error: Could not save summary statistics."

def generate_insights_report(summary_stats_text, shap_analysis_info, kde_plot_desc, violin_plot_desc, parallel_coords_desc, hypergraph_desc, time_series_desc, output_path):
    """Generates a comprehensive insights report, handling potential errors."""
    try:
        grok_insights = (
            analyze_text_with_llm(f"Interpret summary statistics: {summary_stats_text}", MODEL_GROK_NAME) + "\n\n" +
            analyze_text_with_llm(f"Interpret SHAP analysis: {shap_analysis_info}", MODEL_GROK_NAME)
        )
        claude_insights = (
            analyze_text_with_llm(f"Interpret KDE plot: {kde_plot_desc}", MODEL_CLAUDE_NAME) + "\n\n" +
            analyze_text_with_llm(f"Interpret Violin plot: {violin_plot_desc}", MODEL_CLAUDE_NAME) + "\n\n" +
            analyze_text_with_llm(f"Interpret Parallel Coordinates Plot for time series: {parallel_coords_desc}", MODEL_CLAUDE_NAME) + "\n\n" +
            analyze_text_with_llm(f"Interpret Hypergraph: {hypergraph_desc}", MODEL_CLAUDE_NAME) + "\n\n" +
            analyze_text_with_llm(f"Interpret Time Series Line Plot: {time_series_desc}", MODEL_CLAUDE_NAME) + "\n\n"
        )
        grok_enhanced_insights = analyze_text_with_llm(f"Provide enhanced insights on anxiety intervention effectiveness based on time series analysis, SHAP, and Parallel Coordinates, focusing on longitudinal trends.", MODEL_GROK_ENHANCED_NAME)

        combined_insights = f"""
    Combined Insights Report: Longitudinal Anxiety Intervention Analysis

    Grok-base Analysis:
    {grok_insights}

    Claude 3.7 Sonnet Analysis:
    {claude_insights}

    Grok-Enhanced Analysis (Time Series Focused):
    {grok_enhanced_insights}

    Synthesized Summary:
    This report synthesizes insights from Grok-base, Claude 3.7 Sonnet, and Grok-Enhanced, focusing on the longitudinal effects of the anxiety intervention using time series analysis. Grok-base provides a statistical overview and initial interpretations of time-dependent trends, noting the sustained reduction in Group A and the initial reduction followed by a slight increase in Group B. Claude 3.7 Sonnet details visual patterns, including temporal trajectories and distributions over time, highlighting the sustained reduction in anxiety for Group A and the more variable response in Group B.  The parallel coordinates plot is interpreted as showing the individual trajectories. Grok-Enhanced, with a time series focus, delivers nuanced interpretations and actionable recommendations based on longitudinal data, SHAP values, and parallel coordinates, revealing distinct patterns of sustained reduction, initial improvement followed by relapse, and stability across the groups. The combined expert analyses, enhanced by time series techniques, provide a robust and explainable understanding of the intervention's long-term impacts and dynamic effects on anxiety levels across different groups.
    """
        with open(os.path.join(output_path, 'insights.txt'), 'w') as f:
            f.write(combined_insights)
        print(f"Insights saved to: {os.path.join(output_path, 'insights.txt')}")

    except Exception as e:
        print(f"Error generating insights report: {e}")
        print("An error occurred, and the insights report could not be generated.")

# --- Main Script ---
if __name__ == "__main__":
    if not create_output_directory(OUTPUT_PATH):
        exit()

    synthetic_dataset = """
participant_id,group,anxiety_pre,anxiety_post,anxiety_week1,anxiety_week2,anxiety_week3
P001,Group A,4,2,1.8,1.5,1.2
P002,Group A,3,1,0.9,0.7,0.5
P003,Group A,5,3,2.8,2.5,2.3
P004,Group B,6,5,5.2,5.4,5.6
P005,Group B,5,4,4.3,4.5,4.7
P006,Group B,7,6,6.1,6.3,6.5
P007,Control,3,3,3,3,3
P008,Control,4,4,4,4,4
P009,Control,2,2,2,2,2
P010,Control,5,5,5,5,5
"""
    df = load_data_from_synthetic_string(synthetic_dataset)
    is_valid, valid_groups = validate_dataframe(df, [PARTICIPANT_ID_COLUMN, GROUP_COLUMN] + TIME_SERIES_COLUMNS)
    if not is_valid:
        exit()

    # --- DDQN Agent Placeholder ---
    # Example state and action space (adapt to your needs)
    state_dim = len(TIME_SERIES_COLUMNS)  # Example: anxiety levels over time
    action_dim = 3 # Example: no_action, adjust_intervention_A, adjust_intervention_B
    agent = DDQNAgent(state_dim, action_dim)

    # Example usage (replace with actual environment interaction)
    sample_state = df[TIME_SERIES_COLUMNS].iloc[-1].values # Example state (last time series data)
    action = agent.act(np.argmax(sample_state)) # Get action for the state
    print(f"\nDDQN Agent Action (Placeholder): {action}") # Output the action


    # Keep the original group for plots
    df_original_group = df[GROUP_COLUMN].copy()

    df = pd.get_dummies(df, columns=[GROUP_COLUMN], prefix=GROUP_COLUMN, drop_first=False) # One-hot encode, keep all groups
    encoded_group_cols = [col for col in df.columns if col.startswith(f"{GROUP_COLUMN}_")]

    # Add back the original group (with a new name)
    df['original_group'] = df_original_group

    df = scale_data(df, TIME_SERIES_COLUMNS + encoded_group_cols)
    if df is None:
        exit()

    time_series_desc = perform_time_series_analysis(df.copy(), OUTPUT_PATH)

    shap_feature_columns = encoded_group_cols + [ANXIETY_PRE_COLUMN]
    shap_analysis_info = calculate_shap_values(df.copy(), shap_feature_columns, ANXIETY_POST_COLUMN, OUTPUT_PATH)

    kde_plot_desc = create_kde_plot(df, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH, NEON_COLORS[:2])
    violin_plot_desc = create_violin_plot(df, 'original_group', ANXIETY_POST_COLUMN, OUTPUT_PATH, NEON_COLORS) # Use original group
    parallel_coords_desc = create_parallel_coordinates_plot(df, 'original_group', TIME_SERIES_COLUMNS, OUTPUT_PATH, NEON_COLORS) # Use original group
    hypergraph_desc = visualize_hypergraph(df, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH, NEON_COLORS[:2])
    time_series_line_desc = create_time_series_line_plot(df, 'original_group', TIME_SERIES_COLUMNS, OUTPUT_PATH, NEON_COLORS) # Use original group

    bootstrap_ci, normality_message = perform_bootstrap(df[ANXIETY_POST_COLUMN], np.mean)
    print(normality_message)
    summary_stats_text = save_summary(df, bootstrap_ci, OUTPUT_PATH)

    generate_insights_report(summary_stats_text, shap_analysis_info, kde_plot_desc, violin_plot_desc, parallel_coords_desc, hypergraph_desc, time_series_desc, OUTPUT_PATH)

    print("Execution completed successfully - Time Series Analysis Enhanced Notebook.")

Mounted at /content/drive

DDQN Agent Action (Placeholder): 2
Time Series Analysis Placeholder Output:
Time Series Analysis Placeholder Output:
- Average anxiety reduction observed across all groups over time.
- Group A shows a sustained reduction in anxiety levels.
- Group B shows initial reduction, with slight increase in later weeks.
- Control group anxiety levels remain relatively stable.



<ipython-input-1-e400ee92216e>:254: UserWarning: The palette list has more values (4) than needed (3), which may not be intended.
  sns.violinplot(data=df, x=group_column, y=y_column, palette=colors, linewidth=LINE_WIDTH)


Error generating parallel coordinates plot: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

Bootstrap distribution may not be normal.
Insights saved to: /content/drive/MyDrive/output_anxiety_time_series/insights.txt
Execution completed successfully - Time Series Analysis Enhanced Notebook.
